In [1]:
// we need to specify the path and explicitly load the CAF library
#pragma cling add_library_path("/usr/local/lib")
#pragma cling add_include_path("/usr/local/include")
#pragma cling load("/usr/local/lib/libcaf_core.so")

# Testing

In [2]:
// CAF_TEST_NO_MAIN is required for this notebook
#define CAF_TEST_NO_MAIN

// CAF_SUITE specifies the name of the test_suite
// Your can divide your tests in multiple testsuites
// For this notebook we will only one suite
#define CAF_SUITE playground_suite

Necessary header file inclusion

In [3]:
#include <caf/test/dsl.hpp>
#include <caf/test/unit_test.hpp>
#include <caf/test/unit_test_impl.hpp>

## Most basic test case

In [4]:
namespace n1 {

struct fixture {};
    
CAF_TEST_FIXTURE_SCOPE(math_tests, fixture)

CAF_TEST(divide) {
    CAF_CHECK(1 / 1 == 0); // this would fail
    CAF_CHECK(2 / 2 == 1); // this would pass
//     CAF_REQUIRE(3 + 3 == 5); // this would fail and stop test execution [uncomment to try]
    CAF_CHECK(4 - 4 == 0); // You would not reach here because of failed REQUIRE
}

CAF_TEST_FIXTURE_SCOPE_END()
    
}

Above is a simple fixture that contains one test case only i.e. **divide**.

We have used few macros such as CAF_CHECK and CAF_REQUIRE to validate our assertions. The main
difference between CAF_REQUIRE and CAF_CHECK is that even if CAF_CHECK fails the control flow will
continue, however failure of assertion by CAF_REQUIRE will stop the test exeuction.

## Testing Actors

A simple example of a test case is shown below. This example shows that you can create the actor system in your fixture, spawn actors and send messages to them. In other words, below code is not very different from your regular program however here we are using the macros such as CAF_CHECK and have arranged them as test cases.

In [5]:
namespace n2 {
    
#define ERROR_HANDLER [&](caf::error &err) { CAF_FAIL(sys.render(err)); }
    
struct actor_fixture {
  caf::actor_system_config cfg;
  caf::actor_system sys;
  caf::scoped_actor self;
    
  actor_fixture()
      : sys(cfg),
        self(sys) {}

  ~actor_fixture() {}
};

caf::behavior adder(caf::event_based_actor *self) {
    return {
        [=](int x, int y) -> int {
            return x+y;
        }
    };
}    
    
CAF_TEST_FIXTURE_SCOPE(actor_tests, actor_fixture)

CAF_TEST(simple_actor_test) {
    auto adder_actor = sys.spawn(adder);
    
    self->request(adder_actor, caf::infinite, 3, 4).receive([=](int r){
        CAF_CHECK(r == 7);
    }, ERROR_HANDLER);
}

CAF_TEST_FIXTURE_SCOPE_END()
    
}

While the above example works, very soon you would start to face following problems -

* Lot of boilerplate
* Above is a simple example of one actor, if you are unit testing one actor it would work however the reality
is that you would have your actor invoking another actor. Writing code to validate that behavior is not so easy.
* You primary goal would be to check the interaction between the actors and not necessarily the scheduling on 
multiple threads and/or the asynchronous nature of it.

So how do we write the tests in more declarative and synchronous manner ?

### Test Coordinator

CAF provides an implementation of coordinator (called test_coordinator) that you supply to the scheduler. This coordinator is specifically designed for testing as it does not perform/schedule your actors on multiple thread and provide you the means to run it.

There is also a fixture class called *test_coordinator_fixture* that is provided to hide the details and boilerplate for setting up the scheduler with **test_corrdinator**.

In [6]:
namespace n3 {
    
using an_atom =
    caf::atom_constant<caf::atom("an_atom")>;
    
caf::behavior ping(caf::event_based_actor* self) {
    return {
      [=](an_atom) -> std::string {
          return "pong";
      }  
    };
}
    
caf::behavior pong(caf::event_based_actor* self) {
    return {
      [=](an_atom, bool pang) -> std::string {
          return pang ? "pang" : "ping";
      }  
    };
}
    
    
struct ping_pong_fixture : test_coordinator_fixture<> {
};
    
CAF_TEST_FIXTURE_SCOPE(ping_pong_tests, ping_pong_fixture)

CAF_TEST(ping_should_return_pong) {
    auto ping_actor = sys.spawn(ping);
    
    self->send(ping_actor, an_atom::value);
    
    // check if we sent it correctly
    expect((an_atom), from(self).to(ping_actor).with(an_atom::value));
    // check the response we will get back
    expect((std::string), from(ping_actor).to(self).with("pong"));
}
    

CAF_TEST(pong_should_return_ping_or_pang) {
    auto pong_actor = sys.spawn(pong);
    
    // check if we pass true that it should return pang
    self->send(pong_actor, an_atom::value, true);
    
    // check if we sent it correctly
    expect((an_atom, bool), from(self).to(pong_actor).with(an_atom::value, true));
    // check the response we will get back
    expect((std::string), from(pong_actor).to(self).with("pang"));
    
        
    // check if we pass false that it should return ping
    self->send(pong_actor, an_atom::value, false);
    
    // check if we sent it correctly
    expect((an_atom, bool), from(self).to(pong_actor).with(an_atom::value, false));
    // check the response we will get back
    expect((std::string), from(pong_actor).to(self).with("ping"));
    
}

CAF_TEST_FIXTURE_SCOPE_END()
    
}

Above shows an excellent way to declarative testing of your actors.


What happens behind the scenes is that **expect** macro schedules the run using the test_coordinator. Now there will
be scenarios where before you get to test your actor implementation you may want to set them up. That setup would require sending some messages. 

Next example show case the pattern that you could use for such test cases.

In [7]:
namespace n4 {
    
// Here we have a stateful actor that requires that
// you call its third method only after the first and second methods
// have been called
    
struct SomeActorInfo{
    bool invoked_first_method;
    bool invoked_second_method;
};   
    
caf::behavior make_some_actor(caf::stateful_actor<SomeActorInfo> *self) {
    return {
      [=](int x) -> std::string {
          self->state.invoked_first_method = true;
          return "invoked method with int";
      },
        
      [=](float y) -> std::string {
          self->state.invoked_second_method = true;
          return "invoked method with float";
      },
        
      [=](std::string) -> int {
          if (self->state.invoked_first_method == false || 
              self->state.invoked_second_method == false) {
              return -1;
          }
          return 0;
      }
    };
}
    
    
struct some_actor_fixture : test_coordinator_fixture<> {
};
    
CAF_TEST_FIXTURE_SCOPE(some_actor_tests, some_actor_fixture)

CAF_TEST(some_actor_test_3rd_method) {
    auto some_actor = sys.spawn(make_some_actor);
    
    // in this test case we are only interested in
    // testing the third method
    //
    // However the first and second method needs to be invoked
    // as well so that they can update the proper state.
    //
    // While we could using 'expect' macro 2 times for each method call
    // but it would be just noise. Instead we will use 'run' method to
    // dispatch all the pending messages to all the actors
    
    self->send(some_actor, 1);
    self->send(some_actor, (float)0.3);

    std::cout << "Number of messages processed - " << run() << std::endl;
    
    // now we can finally test the method that we wanted to test
    self->send(some_actor, "hey");
    
    expect((std::string), from(self).to(some_actor).with("hey"));
    expect((int), from(some_actor).to(self).with(0));
    
}


CAF_TEST_FIXTURE_SCOPE_END()
    
}

In [ ]:
// This is to run the test suite from the notebook.
// In your actual program you would not need to do 
// this as you would simply run the executable. Make sure
// to not define CAF_TEST_NO_MAIN
char* tn = (char *)std::string("tests").c_str();
caf::test::main(1, &tn);

+----------------------------------------------------------------------+
                            playground_suite
+----------------------------------------------------------------------+

- divide
  -> 3 checks took 1 ms
 (2/1)
- simple_actor_test
  -> 1 check took 240 ms

- ping_should_return_pong
  -> expect(an_atom).from(self).to(ping_actor).with(an_atom::value) [line 33]
  -> expect(std::string).from(ping_actor).to(self).with("pong") [line 35]
  -> 7 checks took 219 ms

- pong_should_return_ping_or_pang
  -> expect(an_atom, bool).from(self).to(pong_actor).with(an_atom::value, true) [line 45]
  -> expect(std::string).from(pong_actor).to(self).with("pang") [line 47]
  -> expect(an_atom, bool).from(self).to(pong_actor).with(an_atom::value, false) [line 54]
  -> expect(std::string).from(pong_actor).to(self).with("ping") [line 56]
  -> 16 checks took 212 ms

- some_actor_test_3rd_method


Number of messages processed - 

  -> first [line 16]
  -> second [line 22]


2


  -> expect(std::string).from(self).to(some_actor).with("hey") [line 62]
  -> third [line 27]
  -> expect(int).from(some_actor).to(self).with(0) [line 63]
  -> Message does not match expected pattern: ("invoked method with int") [line 284]
     REQUIRED: test failure
     /usr/local/include/caf/test/dsl.hpp:354  had last successful check